In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [2]:
from google.colab import files
uploaded=files.upload()


Saving spam.csv to spam.csv


In [0]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense,Input,GlobalMaxPooling1D,LSTM,Embedding
from tensorflow.keras.models import  Model


In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded['spam.csv']),encoding='ISO-8859-1')


In [5]:
df.head()


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [0]:
df=df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1)


In [7]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
df.columns=("Labels","Messege")

In [9]:
df.head()

,Labels,Messege
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [0]:
df['Binary Labels']=df['Labels'].map({'ham':0,'spam':1})

In [11]:
df.head()

,Labels,Messege,Binary Labels
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [0]:
x_train,x_test,y_train,y_test=train_test_split(df['Messege'],df['Binary Labels'],test_size=0.2)


MAX_VOCAB_SIZE=20000
tokenizer=Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(x_train) #don't fit the test inputs
train_sequences=tokenizer.texts_to_sequences(x_train)
test_sequences=tokenizer.texts_to_sequences(x_test)


In [13]:
word2idx = tokenizer.word_index
V = len(word2idx)
print(V)

7938


In [0]:
train_data=pad_sequences(train_sequences)

In [15]:
print(train_data.shape)

(4457, 121)


In [16]:
T=train_data.shape[1]
print(T)

121


In [17]:
test_data=pad_sequences(test_sequences,maxlen=T)
print(test_data.shape)

(1115, 121)


In [0]:
D=22 #Embedding dimensionality,Hyperparameter
M=16 #Hidden state dimensionality,hyperparameter
#Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.

i=Input(shape=(T,))
x=Embedding(V+1,D)(i)
x=LSTM(M,return_sequences=True)(x)
x=GlobalMaxPooling1D()(x)
x=Dense(1,activation='sigmoid')(x)

model=Model(i,x)

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [20]:
r=model.fit(train_data,y_train,validation_data=(test_data,y_test),epochs=10)

Epoch 1/10
140/140 [==============================] - 7s 51ms/step - loss: 0.4442 - accuracy: 0.8618 - val_loss: 0.3463 - val_accuracy: 0.8619
Epoch 2/10
140/140 [==============================] - 7s 47ms/step - loss: 0.2350 - accuracy: 0.8730 - val_loss: 0.1884 - val_accuracy: 0.9058
Epoch 3/10
140/140 [==============================] - 7s 48ms/step - loss: 0.1335 - accuracy: 0.9827 - val_loss: 0.1296 - val_accuracy: 0.9785
Epoch 4/10
140/140 [==============================] - 7s 47ms/step - loss: 0.0778 - accuracy: 0.9930 - val_loss: 0.1003 - val_accuracy: 0.9785
Epoch 5/10
140/140 [==============================] - 7s 49ms/step - loss: 0.0472 - accuracy: 0.9964 - val_loss: 0.0797 - val_accuracy: 0.9821
Epoch 6/10
140/140 [==============================] - 7s 48ms/step - loss: 0.0303 - accuracy: 0.9989 - val_loss: 0.0690 - val_accuracy: 0.9830
Epoch 7/10
140/140 [==============================] - 7s 48ms/step - loss: 0.0205 - accuracy: 0.9996 - val_loss: 0.0665 - val_accuracy: 0.9830